<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{course-cicd} -->

# "Deploy" models with wandb/GitHub

Promote a model to the model registry and document the promotion with the GitHub Deployment API.  

**Note: We aren't actually deploying a model, instead we are promoting a model with the tag `production candidate` so that your deployment systems can take it from there.** The point of this exercise is to use GitHub Actions and W&B to document this process so you have more transparency and visibility. 

In [ ]:
import os, wandb
from urllib.parse import urlencode

assert os.getenv('WANDB_API_KEY'), 'You must set the WANDB_API_KEY environment variable'

Promote a model to the registry

## Parameters

These will we passed as environment variables in GitHub Actions

In [ ]:
owner='hamelsmu'
repo='wandb-cicd'
registry_path = 'av-team/model-registry/BDD Semantic Segmentation'
run_path = 'av-team/mlops-course-001/'
run_id = '2wpuuc70'
pr_num = '16'
environment = 'staging'

### Promote run's model to the registry

In [ ]:
api = wandb.Api()
run = api.run(f'{run_path}/{run_id}')

art = [a for a in run.logged_artifacts() if a.type == 'model']
if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(registry_path, aliases=['production candidate'])

In [ ]:
versions = api.artifact_versions('model', registry_path)
latest_model = versions[0]

In [ ]:
latest_model.entity, latest_model.version, 

('av-team', 'v3')

Get the url for the registry

In [ ]:
query = urlencode({'selectionPath': registry_path, 'version': latest_model.version})
registry_url = f'https://wandb.ai/{latest_model.entity}/registry/model?{query}'
registry_url

'https://wandb.ai/av-team/registry/model?selectionPath=av-team%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v3'

### Share the link on GitHub with deployments

In [ ]:
from ghapi.core import GhApi
gapi = GhApi(owner=owner, repo=repo)

When you are making an issue coment, you need to get the branch associated with the PR with an API call

In [ ]:
branch_name = gapi.pulls.get(pr_num).head.ref

In [ ]:
deploy = gapi.repos.create_deployment(ref=branch_name,
                                      auto_merge=False,
                                 environment=environment,
                                 description='promote model to `deployment candidate`',
                                 payload={'run_id': run_id, 'registry_url':registry_url})

You could actually deploy your model in and report the status of `success` or `failure` back.  However, we aren't going to do that and let another system handle the actual deployment from the model registry.

In [ ]:
status = gapi.repos.create_deployment_status(deployment_id=deploy.id,
                                             description=f'Tagged model for run {run_id} as a `production candidate`',
                                             environment=environment,
                                             log_url=registry_url,
                                             state='success')

In [ ]:
status

```json
{ 'created_at': '2023-01-31T20:52:10Z',
  'creator': { 'avatar_url': 'https://avatars.githubusercontent.com/u/1483922?v=4',
               'events_url': 'https://api.github.com/users/hamelsmu/events{/privacy}',
               'followers_url': 'https://api.github.com/users/hamelsmu/followers',
               'following_url': 'https://api.github.com/users/hamelsmu/following{/other_user}',
               'gists_url': 'https://api.github.com/users/hamelsmu/gists{/gist_id}',
               'gravatar_id': '',
               'html_url': 'https://github.com/hamelsmu',
               'id': 1483922,
               'login': 'hamelsmu',
               'node_id': 'MDQ6VXNlcjE0ODM5MjI=',
               'organizations_url': 'https://api.github.com/users/hamelsmu/orgs',
               'received_events_url': 'https://api.github.com/users/hamelsmu/received_events',
               'repos_url': 'https://api.github.com/users/hamelsmu/repos',
               'site_admin': False,
               'starred_url': 'https://api.github.com/users/hamelsmu/starred{/owner}{/repo}',
               'subscriptions_url': 'https://api.github.com/users/hamelsmu/subscriptions',
               'type': 'User',
               'url': 'https://api.github.com/users/hamelsmu'},
  'deployment_url': 'https://api.github.com/repos/hamelsmu/wandb-cicd/deployments/784106961',
  'description': 'Tagged model for run 2wpuuc70 as a `production candidate`',
  'environment': 'staging',
  'environment_url': '',
  'id': 1628518990,
  'log_url': 'https://wandb.ai/av-team/registry/model?selectionPath=av-team%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v3',
  'node_id': 'DES_kwDOIxeptM5hETpO',
  'performed_via_github_app': None,
  'repository_url': 'https://api.github.com/repos/hamelsmu/wandb-cicd',
  'state': 'success',
  'target_url': 'https://wandb.ai/av-team/registry/model?selectionPath=av-team%2Fmodel-registry%2FBDD+Semantic+Segmentation&version=v3',
  'updated_at': '2023-01-31T20:52:10Z',
  'url': 'https://api.github.com/repos/hamelsmu/wandb-cicd/deployments/784106961/statuses/1628518990'}
```